In [1]:
import pandas as pd

# Задание 1
Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

если источник traffic_source равен yandex или google, то в source_type ставится organic
для источников paid и email из России - ставим ad
для источников paid и email не из России - ставим other
все остальные варианты берем из traffic_source без изменений

In [2]:
log = pd.read_csv('visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [3]:
def source_type(row):
    if (row['traffic_source']=='yandex') | (row['traffic_source']=='google'):
        return 'organic'
    elif ((row['traffic_source']=='paid') | (row['traffic_source']=='email')) & (row['region'] == 'Russia'):     
        return 'ad'
    elif ((row['traffic_source']=='paid') | (row['traffic_source']=='email')) & (row['region'] != 'Russia'):     
        return 'other'        
    else: 
        return row['traffic_source']

In [4]:
log['source_type'] = log.apply(source_type, axis=1)
log.head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,ad
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,direct
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,direct
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,ad


# Задание 2
В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

Прочитайте содержимое файла с датафрейм
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [5]:
url = pd.read_csv('urls.txt')
url.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [6]:
result = url[url.url.str.contains('/\d{8}-')]
result.head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


# Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [7]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [8]:
rating_count = ratings[['userId', 'timestamp']].groupby('userId').agg(['count','min','max']).reset_index()
rating_count['different'] = rating_count['timestamp']['max'] - rating_count['timestamp']['min']
rating_count.head()

userId timestamp                         different
             count         min         max          
0      1        20  1260759108  1260759205        97
1      2        76   835355395   835356246       851
2      3        51  1298861589  1298932787     71198
3      4       204   949778714   949982274    203560
4      5       100  1163373044  1163375145      2101

In [9]:
active_users = rating_count[['userId', 'different']][rating_count['timestamp']['count'] > 100]
active_users.head()

,userId,different
,,
3,4,203560
7,8,85187
14,15,471393496
16,17,8053
18,19,5282


In [10]:
result = active_users.different.mean()
print(result)

40080507.4496124


# Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе Материалы для лекции «Продвинутый pandas» ---- Ноутбуки к лекции «Продвинутый pandas»).
Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [11]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [12]:
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [13]:
table_without_address = rzd.merge(auto, on='client_id', how='outer')
table_without_address = table_without_address.merge(air, on='client_id', how='outer' )
table_without_address.fillna(0)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [14]:
table_with_address = table_without_address.merge(client_base, on='client_id', how='outer' )
table_with_address.fillna(0)

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4
